<a href="https://colab.research.google.com/github/luciaPardo/DemoGithub/blob/main/AnalisysSentimentModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [4]:
import datetime
import shutil

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
# !pip install tensorflow==2.8.0
!pip3 install --quiet "tensorflow-text==2.8.*"
# !pip install tensorflow-text==2.8.2
import tensorflow_text as text
from google.cloud import aiplatform
!git clone https://github.com/tensorflow/models.git

import os
os.environ['PYTHONPATH'] += ":/content/models"

import sys
sys.path.append("/content/models")
# from official.nlp import optimization  # to create AdamW optmizer

tf.get_logger().setLevel("ERROR")

ImportError: /usr/local/lib/python3.10/dist-packages/tensorflow_text/core/pybinds/tflite_registrar.so: undefined symbol: _ZN4absl12lts_2021032420raw_logging_internal21internal_log_functionE

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  1


In [9]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('sentiment_integrado.csv')
df.head(10)

,Unnamed: 0,id,texto,sentimiento,Cantidad palabras,cant caracteres especiales,cant menciones,cant hashtags
0,0,142389495503925248,"Salgo de #VeoTV , que día más largoooooo...",Indefinido,8,4,0,1
1,1,142389933619945473,@PauladeLasHeras No te libraras de ayudar me/n...,Neutral,10,2,1,0
2,2,142391947707940864,@marodriguezb Gracias MAR,Positivo,3,0,1,0
3,3,142416095012339712,"Off pensando en el regalito Sinde, la que se v...",Negativo,24,4,0,0
4,4,142422495721562112,Conozco a alguien q es adicto al drama! Ja ja ...,Positivo,15,2,0,0
5,5,142424715175280640,"RT @FabHddzC: Si amas a alguien, déjalo libre....",Indefinido,16,6,2,0
6,6,142483342040907776,Toca @crackoviadeTV3 . Grabación dl especial N...,Positivo,8,5,1,0
7,7,142493511634259968,Hoy asisitiré en Madrid a un seminario sobre l...,Indefinido,16,1,0,0
8,8,142494476051562496,Buen día todos! Lo primero mandar un abrazo gr...,Positivo,25,2,1,0
9,9,142496796416016384,Desde el escaño. Todo listo para empezar #endi...,Positivo,12,6,0,1


In [14]:

# Elimino los valores nulos de Texto
df_final = df[['sentimiento', 'texto']].dropna(subset=['texto'])

print("Dimensiones de df_final:", df_final.shape)

Dimensiones de df_final: (17069, 2)


In [15]:
# Divide el conjunto de datos en entrenamiento (60%), prueba (20%) y validación (20%)
from sklearn.model_selection import train_test_split
train_df, test_valid_data = train_test_split(df_final, test_size=0.4, random_state=42)
test_df, val_df = train_test_split(test_valid_data, test_size=0.5, random_state=42)

# Imprime el tamaño de cada conjunto
print("Tamaño del conjunto de entrenamiento:", len(train_df))
print("Tamaño del conjunto de prueba:", len(test_df))
print("Tamaño del conjunto de validación:", len(val_df))

Tamaño del conjunto de entrenamiento: 10241
Tamaño del conjunto de prueba: 3414
Tamaño del conjunto de validación: 3414


# Transformo dataset de Pandas a Tensorflow


In [16]:
def pandas_to_tensorflow_dataset(df, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((df['texto'].values, df['sentimiento'].values))
    dataset = dataset.shuffle(buffer_size=len(df)).batch(batch_size)
    return dataset

In [17]:
batch_size = 32

train_ds = pandas_to_tensorflow_dataset(train_df, batch_size=batch_size)
test_ds = pandas_to_tensorflow_dataset(test_df, batch_size=batch_size)
val_ds = pandas_to_tensorflow_dataset(val_df, batch_size=batch_size)

O

In [18]:
## Se obtiene modelo de TensorFlow Hub

In [19]:
# defining the URL of the smallBERT model to use
tfhub_handle_encoder = (
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"
)

# defining the corresponding preprocessing model for the BERT model above
tfhub_handle_preprocess = (
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
)

print(f"BERT model selected           : {tfhub_handle_encoder}")
print(f"Preprocess model auto-selected: {tfhub_handle_preprocess}")

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
